In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Arial']

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [ ]:
data = pd.read_csv('/Users/csuen/radicals/representative regions/UKESM1-0-LL_2010_2014_representative_regions.csv')

In [5]:
list(data)

['index',
 'longitude',
 'latitude',
 't',
 'loc_x',
 'o3',
 'o3prod',
 'o3loss',
 'eminox',
 'emivoc',
 'tas',
 'hus',
 'aod',
 'bldep',
 'ua',
 'va',
 'ps',
 'jno2',
 'photo1d',
 'lossch4',
 'lossco',
 'no2',
 'no',
 'oh',
 'h2o',
 'pan',
 'hno3',
 'co',
 'hcho',
 'ch4',
 'ch3o2_mod',
 'ho2_mod',
 'o1d_mod',
 'o3_mole',
 'no2_mole',
 'no_mole',
 'h2o_mole',
 'co_mole',
 'hcho_mole',
 'oh_mole',
 'n2_mole',
 'o2_mole',
 'lossco_mole',
 'lossch4_mole',
 'o1d_mole',
 'ho2_mole',
 'ch3o2_mole',
 'ch3ooh_mole',
 'o1d_mod_vmr',
 'ho2_mod_vmr',
 'ch3o2_mod_vmr',
 'o1d_vmr',
 'ho2_vmr',
 'ch3o2_vmr',
 'o1d_mod_mole',
 'ho2_mod_mole',
 'ch3o2_mod_mole',
 'o1d_mole_adj',
 'ho2_mole_adj',
 'ch3o2_mole_adj',
 'k1',
 'k2',
 'k3',
 'k4',
 'k5',
 'k6',
 'k7',
 'k8',
 'k9',
 'k9a',
 'k9b',
 'k10',
 'k11',
 'k12',
 'k13',
 'k14',
 'k15',
 'k15a',
 'k15b',
 'k16',
 'pt1',
 'pt2',
 'pt4',
 'pt5',
 'pt6',
 'o3prod_mole',
 'o3loss_mole',
 'pt3',
 'pt7',
 'pt3_adj',
 'pt7_adj',
 'o3prod_mole_adj',
 'o3los

#### Calculate the lumped equivalency concentrations 

In [ ]:
data['pt3'] = data['o3prod_mole_stacked'] - data['pt1'] - data['pt2']
data['etoo_eq_mole'] = data['pt3']/data['k18']/data['no_mole']
data['LER_NOx'] = data['etoo_eq_mole']/data['nox_mole']

In [60]:
data['hno3_mole'] = data['hno3']*1.01325*10**5/data['tas']*7.243*10**7*10**9
data['pan_mole']  = data['pan'] *1.01325*10**5/data['tas']*7.243*10**7*10**9

In [61]:
data['nox_mole'] = data['no_mole']+data['no2_mole']
data['noz_mole'] = data['hno3_mole']+data['pan_mole']
data['noy_mole'] = data['nox_mole']+data['noz_mole']

In [62]:
data['FNR_NO2'] = data['hcho_mole']/data['no2_mole']
data['FNR_NOx'] = data['hcho_mole']/data['nox_mole']
data['FNR_NOy'] = data['hcho_mole']/data['noy_mole']

In [63]:
data['VOC_NOx'] = data['emivoc']/data['eminox']

In [64]:
sens = pd.read_csv('/Users/csuen/Documents/GitHub/ozone-radical/Main Analysis/extended indicators/region sensitivities.csv')

In [ ]:
data['LER'] = data['etoo_eq_mole']/data['no2_mole']
plt.scatter(np.log(data['LER']), np.log(data['FNR_NO2']))

In [ ]:
LER_NOx = data.groupby(['region']).mean()['LER_NOx']

FNR_NO2 = data.groupby(['region']).mean()['FNR_NO2']
FNR_NOx = data.groupby(['region']).mean()['FNR_NOx']
FNR_NOy = data.groupby(['region']).mean()['FNR_NOy']

VOC_NOx = data.groupby(['region']).mean()['VOC_NOx']

In [ ]:
diag = pd.DataFrame(LER)
diag['LER_NOx'] = LER_NOx
diag['FNR_NO2'] = FNR_NO2
diag['FNR_NOx'] = FNR_NOx
diag['FNR_NOy'] = FNR_NOy
diag['VOC_NOx'] = VOC_NOx
diag = diag.merge(sens, how='left', left_on=['region'], right_on=['Region'])
diag